Домашнее задание 2. 
Лаврухина Юлиана, М04-006г.

In [1]:
!apt install sox

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3
Suggested packages:
  file libsox-fmt-all
The following NEW packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3 sox
0 upgraded, 8 newly installed, 0 to remove and 37 not upgraded.
Need to get 760 kB of archives.
After this operation, 6,717 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopencore-amrnb0 amd64 0.1.3-2.1 [92.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopencore-amrwb0 amd64 0.1.3-2.1 [45.8 kB]
Get:3 http://archive.ubuntu.com/ubun

### Baseline commands recognition (2-5 points)

We're now going to train a classifier to recognize voice. More specifically, we'll use the [Speech Commands Dataset] that contains around 30 different words with a few thousand voice records each.

In [2]:
import os
from IPython.display import display, Audio
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import librosa
import torch
from torch.utils.data import TensorDataset, DataLoader

datadir = "speech_commands"

!wget http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz -O speech_commands_v0.01.tar.gz
# alternative url: https://www.dropbox.com/s/j95n278g48bcbta/speech_commands_v0.01.tar.gz?dl=1
!mkdir {datadir} && tar -C {datadir} -xvzf speech_commands_v0.01.tar.gz 1> log

samples_by_target = {
    cls: [os.path.join(datadir, cls, name) for name in os.listdir("./speech_commands/{}".format(cls))]
    for cls in os.listdir(datadir)
    if os.path.isdir(os.path.join(datadir, cls))
}
print('Classes:', ', '.join(sorted(samples_by_target.keys())[1:]))

--2021-10-28 15:06:15--  http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 209.85.147.128, 2607:f8b0:4001:c06::80
Connecting to download.tensorflow.org (download.tensorflow.org)|209.85.147.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1489096277 (1.4G) [application/gzip]
Saving to: ‘speech_commands_v0.01.tar.gz’

speech_commands_v0. 100%[===================>]   1.39G   131MB/s    in 10s     

2021-10-28 15:06:26 (141 MB/s) - ‘speech_commands_v0.01.tar.gz’ saved [1489096277/1489096277]

Classes: bed, bird, cat, dog, down, eight, five, four, go, happy, house, left, marvin, nine, no, off, on, one, right, seven, sheila, six, stop, three, tree, two, up, wow, yes, zero


In [3]:
!sox --info speech_commands/bed/00176480_nohash_0.wav


Input File     : 'speech_commands/bed/00176480_nohash_0.wav'
Channels       : 1
Sample Rate    : 16000
Precision      : 16-bit
Duration       : 00:00:01.00 = 16000 samples ~ 75 CDDA sectors
File Size      : 32.0k
Bit Rate       : 256k
Sample Encoding: 16-bit Signed Integer PCM



In [4]:
from sklearn.model_selection import train_test_split
from itertools import chain
from tqdm import tqdm
import joblib as jl

classes = ("left", "right", "up", "down", "stop")

def preprocess_sample(filepath, max_length=150):
    amplitudes, sr = librosa.core.load(filepath)
    spectrogram = librosa.feature.melspectrogram(amplitudes, sr=sr)[:, :max_length]
    spectrogram = np.pad(spectrogram, [[0, 0], [0, max(0, max_length - spectrogram.shape[1])]], mode='constant')
    target = classes.index(filepath.split(os.sep)[-2])
    return np.float32(spectrogram), np.int64(target)

all_files = chain(*(samples_by_target[cls] for cls in classes))
spectrograms_and_targets = jl.Parallel(n_jobs=-1)(tqdm(list(map(jl.delayed(preprocess_sample), all_files))))
X, y = map(np.stack, zip(*spectrograms_and_targets))
X = X.transpose([0, 2, 1])  # to [batch, time, channels]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

100%|██████████| 11834/11834 [07:38<00:00, 25.78it/s]


In [5]:
X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)

In [6]:
device = 'cuda'

batch_size = 16

tensor_x = torch.Tensor(X_train)
tensor_y = torch.LongTensor(y_train)

train_dataset = TensorDataset(tensor_x, tensor_y)

tensor_x = torch.Tensor(X_test) # transform to torch tensor
tensor_y = torch.LongTensor(y_test)

test_dataset = TensorDataset(tensor_x, tensor_y)


trainloader = DataLoader(train_dataset, batch_size=batch_size,
                         shuffle=True, num_workers=2)
testloader = DataLoader(test_dataset, batch_size=batch_size,
                        shuffle=False, num_workers=2)

In [7]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # TODO: define your layers here
        self.conv1 = nn.Conv2d(1, 64, 5)
        self.bn1 = nn.BatchNorm2d(64)
        self.dropout1 = nn.Dropout(0.2)
        self.conv2 = nn.Conv2d(64, 128, 5)
        self.bn2 = nn.BatchNorm2d(128)
        self.pool1 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(128, 256, 5)
        self.bn3 = nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool2d(4)
        self.conv4 = nn.Conv2d(256, 256, 5)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool3 = nn.MaxPool2d(6)
        self.flatten = nn.Flatten()
        self.dense1 = nn.Linear(512, 256)
        self.dense2 = nn.Linear(256, 5)

    def forward(self, x):
        # TODO: apply your layers here
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.pool2(x)
        x = self.conv4(x)
        x = self.bn4(x)
        x = F.relu(x)
        x = self.pool3(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = F.relu(x)
        x = self.dense2(x)
        x = F.softmax(x)
        return x


net = Net().to(device)

In [8]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [9]:
def get_acc(loader, model):
    num_corr = 0
    num_samp = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_corr += (predictions == y).sum()
            num_samp += predictions.size(0)
    
    model.train()
    return float(num_corr)/float(num_samp)*100

In [10]:
import seaborn as sn
sn.set()
from IPython.display import clear_output

import time
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import trange

from IPython import display
from ipywidgets import Output

In [11]:
def plot_progress(losses, scores, disp):
  with disp:
    fig, ax = plt.subplots(1, 2, figsize=(16, 5))
    ax[0].plot([i*100 for i in range(len(losses))], losses)
    ax[1].plot([i*100 for i in range(len(scores))], scores)
    ax[0].scatter([i*100 for i in range(len(losses))], losses, color='red')
    ax[1].scatter([i*100 for i in range(len(scores))], scores, color='red')


    ax[0].set(xlabel='samples', ylabel='loss',
          title=f'Training loss')
    ax[1].set(xlabel='samples', ylabel='accuracy',
          title=f'Test accuracy')
    clear_output(wait=True)
    plt.show()
  time.sleep(0.5)

In [12]:
losses = []
scores = []
out = Output()
display.display(out)

for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            acc = get_acc(testloader, net)
            print('[%d, %5d] loss: %.3f, accuracy: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100, acc))
            losses.append(running_loss / 100)
            scores.append(acc)
            plot_progress(losses, scores, out)
            running_loss = 0.0

print('Finished Training')

Output()

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[1,   100] loss: 1.601, accuracy: 30.990
[1,   200] loss: 1.581, accuracy: 31.734
[1,   300] loss: 1.583, accuracy: 30.483
[1,   400] loss: 1.575, accuracy: 35.147
[1,   500] loss: 1.558, accuracy: 35.417
[2,   100] loss: 1.543, accuracy: 35.384
[2,   200] loss: 1.537, accuracy: 37.884
[2,   300] loss: 1.526, accuracy: 43.393
[2,   400] loss: 1.519, accuracy: 37.513
[2,   500] loss: 1.495, accuracy: 46.570
[3,   100] loss: 1.471, accuracy: 50.389
[3,   200] loss: 1.470, accuracy: 48.935
[3,   300] loss: 1.482, accuracy: 45.150
[3,   400] loss: 1.471, accuracy: 50.693
[3,   500] loss: 1.463, accuracy: 54.883
[4,   100] loss: 1.465, accuracy: 51.301
[4,   200] loss: 1.421, accuracy: 50.693
[4,   300] loss: 1.404, accuracy: 58.398
[4,   400] loss: 1.397, accuracy: 58.128
[4,   500] loss: 1.400, accuracy: 55.154
[5,   100] loss: 1.366, accuracy: 64.988
[5,   200] loss: 1.366, accuracy: 59.311
[5,   300] loss: 1.360, accuracy: 51.571
[5,   400] loss: 1.351, accuracy: 63.434
[5,   500] loss:

Train a model: finally, lets' build and train a classifier neural network. You can use any library you like. If in doubt, consult the model & training tips below.

__Training tips:__ here's what you can try:
* __Layers:__ 1d or 2d convolutions, perhaps with some batch normalization in between;
* __Architecture:__ VGG-like, residual, highway, densely-connected, MatchboxNet, Dilated convs - you name it :)
* __Batch size matters:__ smaller batches usually train slower but better. Try to find the one that suits you best.
* __Data augmentation:__ add background noise, faster/slower, change pitch;
* __Average checkpoints:__ you can make model more stable with [this simple technique (arxiv)](https://arxiv.org/abs/1803.05407)
* __For full scale stage:__ make sure you're not losing too much data due to max_length in the pre-processing stage!

These are just recommendations. As long as your model works, you're not required to follow them.

Попробуем улучшить полученные результаты с помощью реализации densely-connected и сравним их.

In [13]:
class Dense_Block(nn.Module):
  def __init__(self, in_channels):
    super(Dense_Block, self).__init__()
    self.relu = nn.ReLU(inplace = True)
    self.bn = nn.BatchNorm2d(in_channels)

    self.conv1 = nn.Conv2d(in_channels = in_channels, out_channels = 32, kernel_size = 3, stride = 1, padding = 1)
    self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = 3, stride = 1, padding = 1)
    self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 32, kernel_size = 3, stride = 1, padding = 1)
    self.conv4 = nn.Conv2d(in_channels = 96, out_channels = 32, kernel_size = 3, stride = 1, padding = 1)
    self.conv5 = nn.Conv2d(in_channels = 128, out_channels = 32, kernel_size = 3, stride = 1, padding = 1)
  
  def forward(self, x):
    bn = self.bn(x)
    conv1 = self.relu(self.conv1(bn))
    conv2 = self.relu(self.conv2(conv1))
    # Concatenate in channel dimension
    c2_dense = self.relu(torch.cat([conv1, conv2], 1))
    conv3 = self.relu(self.conv3(c2_dense))
    c3_dense = self.relu(torch.cat([conv1, conv2, conv3], 1))

    conv4 = self.relu(self.conv4(c3_dense))
    c4_dense = self.relu(torch.cat([conv1, conv2, conv3, conv4], 1))

    conv5 = self.relu(self.conv5(c4_dense))
    c5_dense = self.relu(torch.cat([conv1, conv2, conv3, conv4, conv5], 1))

    return c5_dense

In [14]:
class Transition_Layer(nn.Module): 
  def __init__(self, in_channels, out_channels):
    super(Transition_Layer, self).__init__() 

    self.relu = nn.ReLU(inplace = True) 
    self.bn = nn.BatchNorm2d(num_features = out_channels) 
    self.conv = nn.Conv2d(in_channels = in_channels, out_channels = out_channels, kernel_size = 1, bias = False) 
    self.avg_pool = nn.AvgPool2d(kernel_size = 2, stride = 2, padding = 0) 

  def forward(self, x): 
    bn = self.bn(self.relu(self.conv(x))) 
    out = self.avg_pool(bn) 
    return out

import torch.nn as nn
import torch.nn.functional as F

In [15]:
class NetDense(nn.Module):
    def __init__(self):
        super().__init__()
        # TODO: define your layers here
        self.conv1 = nn.Conv2d(1, 64, 5)
        self.bn1 = nn.BatchNorm2d(64)
        self.dropout1 = nn.Dropout(0.2)
        self.conv2 = nn.Conv2d(64, 128, 5)
        self.bn2 = nn.BatchNorm2d(128)
        self.pool1 = nn.MaxPool2d(2)
        # Make Dense Blocks 
        self.denseblock1 = self._make_dense_block(Dense_Block, 128) 
        self.denseblock2 = self._make_dense_block(Dense_Block, 160)
        self.denseblock3 = self._make_dense_block(Dense_Block, 160)
        # Make transition Layers 
        self.transitionLayer1 = self._make_transition_layer(Transition_Layer, in_channels = 160, out_channels = 128) 
        self.transitionLayer2 = self._make_transition_layer(Transition_Layer, in_channels = 128, out_channels = 128) 
        self.transitionLayer3 = self._make_transition_layer(Transition_Layer, in_channels = 128, out_channels = 128)
        self.conv3 = nn.Conv2d(128, 256, 5)
        self.bn3 = nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool2d(3)
        # self.conv4 = nn.Conv2d(256, 128, 5)
        # self.bn4 = nn.BatchNorm2d(128)
        # self.pool3 = nn.MaxPool2d(6)
        self.flatten = nn.Flatten()
        self.dense1 = nn.Linear(256, 64)
        self.dense2 = nn.Linear(64, 5)

    def _make_dense_block(self, block, in_channels): 
      layers = [] 
      layers.append(block(in_channels)) 
      return nn.Sequential(*layers) 
    def _make_transition_layer(self, layer, in_channels, out_channels): 
      modules = [] 
      modules.append(layer(in_channels, out_channels)) 
      return nn.Sequential(*modules) 

    def forward(self, x):
        # TODO: apply your layers here
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = self.denseblock1(x)
        x = self.denseblock2(x)
        x = self.denseblock3(x)
        x = self.transitionLayer1(x)
        x = self.transitionLayer2(x)
        x = self.transitionLayer3(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.pool2(x)
        # x = self.conv4(x)
        # x = self.bn4(x)
        # x = F.relu(x)
        # x = self.pool3(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = F.relu(x)
        x = self.dense2(x)
        x = F.softmax(x)
        return x

In [16]:
net_dense = NetDense().to(device)

In [19]:
losses = []
scores = []
out = Output()
display.display(out)

criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(net_dense.parameters(), lr=0.001)
optimizer = optim.SGD(net_dense.parameters(), lr=0.001, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in tqdm(enumerate(trainloader, 0), total=len(trainloader)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = net_dense(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            acc = get_acc(testloader, net_dense)
            print('[%d, %5d] loss: %.3f, accuracy: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100, acc))
            losses.append(running_loss / 100)
            scores.append(acc)
            plot_progress(losses, scores, out)
            running_loss = 0.0
    scheduler.step()

print('Finished Training')

Output()

 18%|█▊        | 99/555 [00:45<03:27,  2.19it/s]

[1,   100] loss: 1.570, accuracy: 34.099


 36%|███▌      | 199/555 [01:48<02:40,  2.21it/s]

[1,   200] loss: 1.485, accuracy: 46.840


...

[10,   400] loss: 0.951, accuracy: 63.839


 90%|████████▉ | 499/555 [05:00<00:25,  2.21it/s]

[10,   500] loss: 0.946, accuracy: 67.996


100%|██████████| 555/555 [05:44<00:00,  1.61it/s]

Finished Training


Полученный результат стал немного лучше первоначальнойй модели.

### Full scale commands recognition (3+ points)

Your final task is to train a full-scale voice command spotter and apply it to a video:
1. Build the dataset with all 30+ classes (directions, digits, names, etc.)
  * __Optional:__ include a special "noise" class that contains random unrelated sounds
  * You can download youtube videos with [`youtube-dl`](https://ytdl-org.github.io/youtube-dl/index.html) library.
2. Train a model on this full dataset. Kudos for tuning its accuracy :)
3. Apply it to a audio/video of your choice to spot the occurences of each keyword
 * Here's one [video about primes](https://www.youtube.com/watch?v=EK32jo7i5LQ) that you can try. It should be full of numbers :)
 * There are multiple ways you can analyze the performance of your network, e.g. plot probabilities predicted for every time-step. Chances are you'll discover something useful about how to improve your model :)


Please briefly describe what you did in a short informal report.

Попробуем натренировать модель, которая будет пригодна для всех классов. Будем испольовать BCResNet. 

In [20]:
%cd /content/
!rm -r ./speech_commands

datadir = "speech_commands"

!wget http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz -O speech_commands_v0.01.tar.gz
# alternative url: https://www.dropbox.com/s/j95n278g48bcbta/speech_commands_v0.01.tar.gz?dl=1
!mkdir {datadir} && tar -C {datadir} -xvzf speech_commands_v0.01.tar.gz 1> log

samples_by_target = {
    cls: [os.path.join(datadir, cls, name) for name in os.listdir("./speech_commands/{}".format(cls)) if 'README.md' not in name]
    for cls in os.listdir(datadir)
    if os.path.isdir(os.path.join(datadir, cls))
}
classes = list(samples_by_target.keys())
print('Classes:', ', '.join(sorted(classes)))

/content
--2021-10-28 17:25:01--  http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.202.128, 2607:f8b0:4001:c06::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.202.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1489096277 (1.4G) [application/gzip]
Saving to: ‘speech_commands_v0.01.tar.gz’

speech_commands_v0. 100%[===================>]   1.39G   227MB/s    in 6.5s    

2021-10-28 17:25:08 (218 MB/s) - ‘speech_commands_v0.01.tar.gz’ saved [1489096277/1489096277]

Classes: _background_noise_, bed, bird, cat, dog, down, eight, five, four, go, happy, house, left, marvin, nine, no, off, on, one, right, seven, sheila, six, stop, three, tree, two, up, wow, yes, zero


In [ ]:
def pre_sample(filepath, max_length=160):
    amplitudes, sr = librosa.core.load(filepath)
    spectrogram = librosa.feature.melspectrogram(amplitudes, sr=sr)[:, :max_length]
    spectrogram = np.pad(spectrogram, [[0, 0], [0, max(0, max_length - spectrogram.shape[1])]], mode='constant')
    target = classes.index(filepath.split(os.sep)[-2])
    return np.float32(spectrogram), np.int64(target)

all_files = chain(*(samples_by_target[cls] for cls in classes))

spectrograms_and_targets = jl.Parallel(n_jobs=-1)(tqdm(list(map(jl.delayed(preprocess_sample), all_files))))
X, y = map(np.stack, zip(*spectrograms_and_targets))
X = X.transpose([0, 2, 1])  # to [batch, time, channels]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

100%|██████████| 64727/64727 [41:34<00:00, 25.95it/s]


In [1]:
X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)

if torch.cuda.is_available:
  device = 'cuda'
else:
  device = 'cpu'

batch_size = 16

tensor_x = torch.Tensor(X_train)
tensor_y = torch.LongTensor(y_train)

train_dataset = TensorDataset(tensor_x, tensor_y)

tensor_x = torch.Tensor(X_test) # transform to torch tensor
tensor_y = torch.LongTensor(y_test)

test_dataset = TensorDataset(tensor_x, tensor_y)


trainloader = DataLoader(train_dataset, batch_size=batch_size,
                         shuffle=True, num_workers=2)
testloader = DataLoader(test_dataset, batch_size=batch_size,
                        shuffle=False, num_workers=2)

NameError: ignored

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # TODO: define your layers here
        self.conv1 = nn.Conv2d(1, 64, 5)
        self.bn1 = nn.BatchNorm2d(64)
        self.dropout1 = nn.Dropout(0.2)
        self.conv2 = nn.Conv2d(64, 128, 5)
        self.bn2 = nn.BatchNorm2d(128)
        self.pool1 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(128, 256, 5)
        self.bn3 = nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool2d(4)
        self.conv4 = nn.Conv2d(256, 256, 5)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool3 = nn.MaxPool2d(6)
        self.flatten = nn.Flatten()
        self.dense1 = nn.Linear(512, 256)
        self.dense2 = nn.Linear(256, 31)

    def forward(self, x):
        # TODO: apply your layers here
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.pool2(x)
        x = self.conv4(x)
        x = self.bn4(x)
        x = F.relu(x)
        x = self.pool3(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = F.relu(x)
        x = self.dense2(x)
        x = F.softmax(x)
        return x


net = Net().to(device)

In [ ]:
def get_acc(loader, model):
    num_corr = 0
    num_samp = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_corr += (predictions == y).sum()
            num_samp += predictions.size(0)
       
    model.train()
    return float(num_correct)/float(num_samples)*100

In [ ]:
import seaborn as sn
sn.set()
from IPython.display import clear_output, display

import time
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import trange

from ipywidgets import Output

def plot_progress(losses, scores, disp):
  with disp:
    fig, ax = plt.subplots(1, 2, figsize=(16, 5))
    ax[0].plot([i*100 for i in range(len(losses))], losses)
    ax[1].plot([i*100 for i in range(len(scores))], scores)
    ax[0].scatter([i*100 for i in range(len(losses))], losses, color='red')
    ax[1].scatter([i*100 for i in range(len(scores))], scores, color='red')


    ax[0].set(xlabel='samples', ylabel='loss',
          title=f'Training loss')
    ax[1].set(xlabel='samples', ylabel='accuracy',
          title=f'Test accuracy')
    clear_output(wait=True)
    plt.show()
  time.sleep(1)

In [ ]:
losses = []
scores = []
out = Output()
display.display(out)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in tqdm(enumerate(trainloader, 0), total=len(trainloader)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            acc = get_acc(testloader, net)
            print('[%d, %5d] loss: %.3f, accuracy: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100, acc))
            losses.append(running_loss / 100)
            scores.append(acc)
            plot_progress(losses, scores, out)
            running_loss = 0.0

print('Finished Training')

Память ОЗУ закончилась, поэтому дотренировать модель не получится. Требуются дополнительные ресурсы для дальнейшего исследования. И исходя из полученных результатов рассматривать модели далее.